<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
#импортируем необходимые библиотеки
import pandas as pd

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import numpy as np

RANDOM_STATE = 121345

In [2]:
#читаем и выводим первые 5 строк посмотреть
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
#смотрим общую инфу
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


159к строк, пропусков нет

In [4]:
#смотрим на кол-во явных дубликатов, если они есть
df.duplicated().sum()

0

In [5]:
#грузим лексическую базу данных
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
%%time

#инициализируем лемматизатор
lemmatizer = WordNetLemmatizer()

#пишем функцию, которая поможет нам лемматизировать комментарии
def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    lemmas = []
    for token, pos in pos_tags:
        wn_pos = get_wordnet_pos(pos)
        lemmas.append(lemmatizer.lemmatize(token, pos=wn_pos))
    lemmatized_text = ' '.join(lemmas)
    return lemmatized_text

#функция для преобразования POS-тега в формат WordNet
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return 'a'  
    elif pos_tag.startswith('V'):
        return 'v'  
    elif pos_tag.startswith('N'):
        return 'n'  
    elif pos_tag.startswith('R'):
        return 'r'  
    else:
        return 'n'  
    
df['text'] = df['text'].apply(lemmatize_text)

#посмотрим, что получилось
df.head()

CPU times: user 8min 58s, sys: 4.78 s, total: 9min 2s
Wall time: 9min 3s


,text,toxic
0,Explanation Why the edits make under my userna...,0
1,D'aww ! He match this background colour I 'm s...,0
2,"Hey man , I 'm really not try to edit war . It...",0
3,`` More I ca n't make any real suggestion on i...,0
4,"You , sir , be my hero . Any chance you rememb...",0


In [7]:
#напишем функцию для очистки лемматизированных комментариев от спец. символов
def clear_text(text):
    retext = text.lower()
    retext = re.sub(r'[^a-zA-Z]', ' ', retext)
    retext = retext.split()
    return " ".join(retext)

df['text'] = df['text'].apply(clear_text)

#посмотрим, что получилось
df.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i ca n t make any real suggestion on impr...,0
4,you sir be my hero any chance you remember wha...,0


Вывод: 
1. Пропусков и дубликатов не выявлено.
2. Проведена лемматизация комментариев 
3. Проведена очистка от спец символов

## Обучение

In [8]:
#список стоп-слов английского
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
#выделим признаки и целевой признак 

features = df.drop(['toxic'], axis=1)
target = df['toxic']



#разобьем признаки и целевой признак на 2 группы, отделив обучающие данные от тестовых + валидационных
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.40, random_state=RANDOM_STATE, stratify=target)

#затем разобьем вторую группу пополам: одни данные будут валидационными, другие - тестовыми 
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.50, random_state=RANDOM_STATE, stratify=target_valid_test)

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['text'])
features_valid = count_tf_idf.transform(features_valid['text'])
features_test = count_tf_idf.transform(features_test['text'])

#проверим
display(features_train.shape)
display(features_valid.shape)
display(features_test.shape)
display(target_train.shape)
display(target_valid.shape)
display(target_test.shape)

(95575, 119153)

(31858, 119153)

(31859, 119153)

(95575,)

(31858,)

(31859,)

Обучим логистическую регрессию

In [10]:
%%time

model_lr = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')

param_grid = [{
    'penalty': ['l1', 'l2'],
    'C': list(range(1,15,3)),
    'random_state': [RANDOM_STATE]
}] 


grid = GridSearchCV(model_lr, param_grid=param_grid, scoring='f1', cv=3, verbose=True)
best_grid = grid.fit(features_train, target_train)
display("Наилучшие гиперпараметры:", grid.best_params_)
display("F1-метрика:", grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


'Наилучшие гиперпараметры:'

{'C': 4, 'penalty': 'l1', 'random_state': 121345}

'F1-метрика:'

0.7677052274970287

CPU times: user 1min 19s, sys: 1min 12s, total: 2min 31s
Wall time: 2min 31s


Обучим случайный лес

In [12]:
%%time

params_forest = {
    'n_estimators': [1,50, 100],
    'max_depth': [1,10,50],
    'random_state':[RANDOM_STATE]
}

model_forest = RandomForestClassifier(random_state=RANDOM_STATE)
                                 
grid = GridSearchCV(model_forest, param_grid=params_forest, scoring='f1', cv=3, verbose=True)
best_grid = grid.fit(features_train, target_train)
display("Наилучшие гиперпараметры:", grid.best_params_)
display("F1-метрика:", grid.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


'Наилучшие гиперпараметры:'

{'max_depth': 50, 'n_estimators': 1, 'random_state': 121345}

'F1-метрика:'

0.27614748821703833

CPU times: user 5min 46s, sys: 1.13 s, total: 5min 47s
Wall time: 5min 48s


Обучим дерево решений

In [13]:
%%time

params_tree = {
    'max_depth': [1,10],
    'random_state':[RANDOM_STATE]
}

model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE)
                                 
grid = GridSearchCV(model_tree, param_grid=params_tree, scoring='f1', cv=3, verbose=True)
best_grid = grid.fit(features_train, target_train)
display("Наилучшие гиперпараметры:", grid.best_params_)
display("F1-метрика:", grid.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


'Наилучшие гиперпараметры:'

{'max_depth': 10, 'random_state': 121345}

'F1-метрика:'

0.587727848937947

CPU times: user 31.3 s, sys: 146 ms, total: 31.4 s
Wall time: 31.5 s


## Выводы

Лучшее значение f1 дала модель логистической регрессии, поэтому ее и протестируем на тестовых данных

In [14]:
model_lr = LogisticRegression(C=4, penalty='l1', random_state=RANDOM_STATE, solver='liblinear')
model_lr.fit(features_train, target_train)
predict_lr = model_lr.predict(features_test)
f1_lr = f1_score(predict_lr, target_test)
f1_lr

0.776563033823027

Вывод: 

На первом этапе мы загрузили и подготовили данные для обучения моделей. Затем мы очистили и лемматизировали тексты комментариев.

Для моделирования мы использовали Логистическую регрессию, Случайный лес и Дерево решений. После подбора гиперпараметров и обучения моделей, модель логистической регрессии показала наилучший результат, а Случайный лес - наихудший.

В связи с этим, как решение для данной задачи, модели Логистическая регрессия подходит. На тестовой выборке логистическая регрессия показала значени метрики f1 ~ 0.78.

p.s.
Попробовал только самые легкие модели, т.к. сейчас нет к сожалению возможности работать локально, а на сайте от бустинговых моделей ядро умирает. Берта тоже к сожалению не могу сейчас опробовать.